# Multiclass classification

When solving a classification problem, many existing machine learning models allow only two classes to be separated, usually referred to as "positive" and "negative". Positive patterns are commonly those related to what is to be detected, such as disease, alarm, or a type of object in an image. Negative patterns are often characterised by the absence of this characteristic that positive patterns have. To develop an ANN that classifies into two classes, a single neuron is needed in the output layer, with a logarithmic (or similar) sigmoidal transfer function, such that the output of the ANN will be between 0 and 1, and can be interpreted as the ANN's certainty in classifying a pattern as "positive". The classification into "negative" or "positive" is done in a simple way, by applying a threshold which is typically 0.5, although this can be changed.

However, there are many occasions when a system that is able to classify into more than two classes is desired. A simple example is a system that wants to classify an image according to whether a dog, cat or mouse, or some other type of animal is observed. In this case, you want to develop a 4-class classification system: "dog"/"cat"/"mouse"/"other". If an ANN to distinguish between these 3 animals is required, an output neuron for each class is needed, including the "other" class (4 output neurons in total).

In the multiclass classification scheme, as has been done in previous assignments, an encoding called one-hot-encoding is generally used, which is based on obtaining a boolean value for each pattern and each class, in such a way that each boolean value will be equal to 1 if that pattern belongs to that class, and 0 otherwise. When training an ANN with this scheme, each output neuron can be understood as a model specialised in classifying in a given class. In this type of networks, a linear transfer function is usually used in the output layer, whereby negative outputs indicate that a neuron does not classify the pattern into that class (i.e. from the point of view of that class it classifies it as "negative"), and positive outputs indicate that a neuron classifies the pattern as that class (i.e. from the point of view of that class it classifies it as "positive"). The absolute value of a neuron's output indicates that neuron's confidence in the classification. Finally, the softmax function receives these classification values and transforms them in such a way that they are between 0 and 1, and add up to 1, interpreted as the probability of belonging to each class. The pattern will be classified into the class whose output value is the highest. The softmax function is defined as follows: 

$$
softmax(y^i) = \frac{e^{y^i}}{\sum_j{e^{y^j}}}
$$

where $y^i$ is the output of the $i$-th neuron. For example, in a 3-class classification problem, if the outputs from the 3 neurons are `[2, 1, 0.2]`, they would classify the inputs as belonging to their respective classes, although the first one with much greater certainty. After applying the softmax function, the respective probabilities will be `[0.65, 0.24, 0.11]`, so the pattern will be classified as the first class.

In this way, the softmax function converts the real values produced by the output neurons into probability values, so that the more negative a value is (the more certainty of not belonging to that class), the closer it is to 0, and the more positive a value is (the more certainty of belonging to that class), the closer it is to 1. As indicated above, the sum of output probabilities will be equal to 1. Because of this fact, a fourth special class "other" is needed in the example above and in any other example where a pattern may not belong to any of the predefined classes.

### Question 4.2

> ❓ Why is this extra class necessary when using the softmax function?

The additional "other" class is necessary because the `softmax` function always produces positive probabilities that sum to 1, forcing the network to assign each pattern to one of the available classes. Without this extra class, patterns that do not belong to any of the predefined categories would still be classified as one of them, even if incorrectly. Including an “other” class provides a way for the model to represent such cases appropriately, ensuring a more realistic and robust classification.

Tip: write in Julia `softmax([-1, -1, -0.2])`, and interpret the inputs (what does the vector `[-1, -1, -0.2]` represent and how is it interpreted?) and outputs of the function (how much do the values add up to? what does each say?). To use this function, import it from the Flux library.

In [1]:
using Flux

y = [-1, -1, -0.2]
p = softmax(y)

println(p)
println("Sum of probabilities: ", sum(p))

[0.23665609135556676, 0.23665609135556676, 0.5266878172888664]
Sum of probabilities: 0.9999999999999999


### Question 4.3

> ❓ Might it not be necessary to create the additional class? What modification would have to be made to the ANN? How would the output be interpreted? How would the output class be generated based on the outputs of the output neurons?

It would not be necessary to add the extra class if the ANN did not use the softmax function. Instead, each output neuron could have a linear or sigmoidal activation, producing independent confidence scores for each class. The output class would be the one with the highest value, provided it exceeds a chosen threshold; otherwise, the pattern would be considered as not belonging to any class.

### Question 4.4

> ❓ In general, how does the output of a model have to be in order not to need this fourth class?

In general, the model’s outputs must be independent and not normalized to sum to 1 (i.e., without using the softmax function). Each output neuron should provide an individual confidence value for its class, typically using a sigmoid or linear activation. This way, low values for all outputs naturally indicate that the input does not belong to any known class, making the extra “other” class unnecessary.

### Question 4.5

> ❓ Does a kNN model need this fourth class?

No, a kNN model does not require this fourth class. Unlike a neural network with a softmax output, kNN does not produce normalized probabilities or distribute membership among all classes. Instead, it classifies a new sample by comparing it directly with the training data and assigning it to the majority class among its k nearest neighbors. This means the model always decides the class based on similarity to previous examples, without needing an additional class to represent unknown or ambiguous cases.

### Question 4.6

> ❓ How many classes would be necessary if an ANN wanted to recognise those 3 types of animals, and, if it is not one of them, to say whether it is an animal or not? What if the model is a kNN?

If the model is an ANN, it would need five classes: one for each of the three specific animals (“dog”, “cat”, “mouse”), one for “other animals”, and one for “non-animal”. This structure allows the network, through the softmax output, to assign probabilities that sum to 1 and to distinguish both between known animal types and between animals and non-animals.

If the model is a kNN, it would only require the classes present in the training data. The model does not need to define additional classes in advance, as it classifies new samples based on their similarity to existing labeled examples. Therefore, it would correctly distinguish among the classes only if examples of “other animals” and “non-animals” were also included in the dataset.

Therefore, the "positive"/"negative" scheme no longer applies if more than two classes are required. The problem in these cases is that many of the machine learning models are only capable of separating two classes, so theoretically they could not be used. An example of such systems are Support Vector Machines (SVM), which are discussed in more detail in the theory class. Modifications have been made to the formulation of this model to allow multi-class classifications; however, in practice they are not commonly used, and instead a strategy that allows binary SVMs to be used to classify into multiple classes is often employed.

There are two main strategies for converting multi-class problems into binary classification problems. These strategies are called "one-against-one" or "one-against-all". Both are explained in theory class, but since "one-against-all" is much more widely used, this strategy will be used in the following.

The "one-against-all" strategy is based on generating L binary classifiers for a classification problem of L classes, one per class. In the l-th problem, class l must be separated from the rest, i.e., the patterns belonging to that class will be considered "positive", and those not belonging to it will be considered "negative". Continuing with the previous example of animals, 3 different classification problems would have to be solved: one to classify "dog"/"not dog", one to classify "cat"/"not cat", and one to classify "mouse"/"not mouse". Three classifiers would therefore be trained with the same inputs but with different desired outputs for each problem.

### Question 4.7

> ❓ In the previously described problem, 4 classes were used for these 3 animals, including the class "other". Why not train a classifier for this class in a "one-against-all" scheme?

In a one-against-all scheme, the “other” class is not trained as a separate classifier because it does not represent a single, well-defined category. Instead, it includes all patterns that do not belong to any of the specific classes (“dog”, “cat”, or “mouse”). Training a classifier for “other” would be problematic, since this class could contain a wide variety of unrelated examples (e.g., birds, cars, trees, etc.), making it highly heterogeneous and difficult for the model to learn a consistent decision boundary.

Therefore, in this scheme, only the well-defined classes are used to train separate binary classifiers, while any pattern not classified as belonging to one of them is implicitly assigned to the “other” class.

Once the binary classifiers are trained, any given pattern is fed into all the classifiers and, depending on the output, a decision is made. If only one of the systems has positive output, or none of the three classifies it as positive, the decision is clear. However, sometimes more than one classifier will give a positive output for the same pattern. Fortunately, many classifiers give information about the level of certainty or confidence they have that the pattern is classified as "positive". If more than one binary model classifies the pattern as positive, it will be assigned to the class corresponding to the classifier that has a higher certainty in its classification.

### Question 4.8

> ❓ Would it be possible to use the outputs of those 3 classifiers as the input of the softmax function? What would be the consequences?

Using the outputs of those three classifiers as the input to a softmax function would change the interpretation of the results and could lead to incorrect classifications.

The softmax function assumes that its inputs represent mutually exclusive classes whose probabilities must sum to 1. However, in the one-against-all scheme, each classifier is trained independently, and their outputs are not necessarily comparable or normalized. As a result, applying softmax would force the probabilities to add up to 1 even if multiple classifiers gave strong positive responses, artificially reducing some probabilities and inflating others.

In practice, this could cause ambiguities or distortions in the classification, especially if more than one classifier is confident. Therefore, instead of applying softmax, the correct approach is to directly select the class corresponding to the classifier with the highest confidence value.

### Question 4.9

> ❓ In general, when there are L classes and a pattern may not belong to any of them, what is the impact of using the softmax function on the outputs? In which cases could it be used? Why?

When there are L classes and a pattern may not belong to any of them, using the softmax function forces the model to assign a probability distribution that sums to 1, meaning the pattern will always be classified as belonging to one of the existing classes, even if it does not actually fit any. This can lead to incorrect or overconfident classifications for unknown patterns.

The softmax function can still be used effectively when the problem is closed-set, i.e., when it is guaranteed that every input belongs to one and only one of the predefined classes. In such cases, softmax provides a meaningful probabilistic interpretation of class membership. However, when dealing with open-set problems (where inputs may fall outside the known classes), it is preferable to use independent outputs or threshold-based decisions instead of softmax, to allow the model to represent uncertainty or reject unfamiliar inputs.

### Question 4.10

> ❓ The softmax function is useful to get a loss value to train the ANN. However, if it were not used in the animal example above, would the fourth class "other" be necessary?

If the softmax function were not used, the fourth class “other” would no longer be necessary. Without softmax, the output neurons would be independent, each producing a confidence value (for example, through a sigmoid activation) indicating how strongly the input belongs to its corresponding class.

In this case, if all outputs are low, the network can naturally interpret that the pattern does not belong to any of the known classes, without needing an explicit “other” category. The model could simply apply a threshold to decide whether any class is activated; if none exceeds it, the pattern would be considered as “none of the above.”

Finally, it is necessary to consider a different scenario when assigning patterns to classes. So far, and in most situations, the classes considered are mutually exclusive, i.e. in the example above, an animal is either a dog, a cat, a mouse, or none of the 3, but it cannot be of several classes at the same time. This is the most common case, but occasionally a problem will have classes that are not mutually exclusive. For example, when classifying animal sounds according to the animal that makes them, it may happen that several animals are mixed in one sound. In these cases, the use of a linear transfer function in the last layer together with the softmax function would not work, since, naturally, the sum of the probabilities of belonging to the classes may be greater than 1 (it may belong to several classes at the same time). For these cases, the scheme that can be used to train ANNs is to use logarithmic sigmoidal transfer functions in the last layer (instead of linear), which give an output between 0 and 1, and not to perform transformation using the softmax function. In this way, the final output of each output neuron is independent of the rest of the output neurons, and more than one can take values close to 1. The output of each neuron would again be interpreted as the probability of belonging to that class, but in this case the sum of the probabilities does not have to be 1 (they are independent). Not applying the softmax function has two advantages: the first, already mentioned, is that it allows classification into non-mutually exclusive classes; the second is that an additional class ("other" in the example above) is no longer needed for cases where a set of inputs may not belong to any of the given classes.

### Question 4.11

> ❓ Why is this extra class no longer needed?

This extra class is no longer needed because, without using the softmax function, each output neuron provides an independent probability of belonging to its corresponding class. The outputs are not forced to sum to 1, so the network can represent cases where a pattern belongs to several classes at once or to none of them. If all output values are low, the model naturally interprets that the input does not fit any known class, making an explicit “other” class unnecessary.

Given a set of inputs, as always, it is classified into the class whose output neuron has shown the highest confidence. This scheme of non-mutually exclusive outputs is similar to the "one-against-all" scheme, in which one classifier per class is trained in parallel. The classifiers are independent and the final class is that of the classifier that has the highest certainty of belonging to that class. If all classifiers return "negative" as a classification and there is no possibility of not belonging to any class, the classifier with the lowest certainty of being negative is classified in the corresponding class. If all classifiers return "negative" as a classification and there is a possibility of non-class membership, it is simply classified as "other".

The following table shows a summary of the different scenarios when using an ANN to solve a classification problem. Note that in the case of binary classification, the possibility that a set of entries do not belong to any class is not considered, since in this case we would be in multi-class classification.



|                        |                       | Is extra<br>class added<br>if it can<br>not belong<br>to any of<br>the given? | Transfer<br>function<br>of the<br>output | Output<br>range<br>of the<br>neuron | Function<br>applied<br>to the<br>outputs<br>(additional layer) | Final<br>output<br>range | Which classes<br>it classifies<br>into? |
|------------------------|-----------------------|------------------------------------------|-----------------------------------------|-----------------------------------|----------------------------------------------------|------------------------|------------------------------------|
| **Binary<br>classification** |                       | –                                        | Logarithmic<br>sigmoid                  | [0,1]                             | –                                                  | [0,1]                 | "negative"/"positive" if the output<br>is greater or equal to a threshold<br>(0.5 for outputs in [0,1]) |
| **Multiclass**         | Mutually<br>exclusive<br>classes | Yes                                      | Linear                                  | [-inf, +inf]                      | softmax                                            | [0,1]                 | Class whose output is<br>closer to 1                 |
|                        | Not<br>mutually<br>exclusive<br>classes | No                                       | Logarithmic<br>sigmoid                  | [0,1]                             | –                                                  | [0,1]                 | Classes whose output<br>is greater or equal<br>to a threshold (0.5 for outputs<br>in [0,1]); can be several or none |


In the case of using a "one-against-all" strategy, this would be similar to the last row, except that the interval would not necessarily be `[0, 1]`, but would be conditioned by the model used, and therefore the threshold as well. For example, the outputs of a SVM range from $-\infty$ to $+\infty$, so the typical threshold is set to 0.

Another factor to consider when dealing with multiclass problems is the performance metric. Most of the metrics studied (PPV, sensitivity, etc.) correspond to binary classification problems. When the number of classes is greater than 2, these metrics can still be used; however, their use is slightly different.

When the number of classes is greater than two, the PPV, NPV, sensitivity and specificity metrics can be calculated separately for each class. Thus, from the point of view of a particular class, that class will be referred to as the positive class and the rest of classes will be put together in the negative class. In this way, from the exclusive point of view of that class, TP, TN, FP and FN can be calculated, and from them the sensitivity, specificity, PPV and NPV values for that particular class, and finally the F-score value. This way of treating classes separately is similar to the development of several classifiers in the "one-against-all" strategy (in the case of training binary classifiers that do not allow multi-class classification). Once these values have been calculated, they can be combined into a single value that will be used to evaluate the performance of the classifier. In this regard, there are 3 strategies: macro, weighted, and micro. We will use only the first two:

- **Macro**. In this strategy, those metrics such as the PPV or the F-score are calculated as the arithmetic mean of the metrics of each class. As it is an arithmetic average, it does not consider the possible imbalance between classes.
- **Weighted**. In this stratey, the metrics corresponding to each class are averaged, weighting them with the number of patterns that belong (desired output) to each class. It is therefore suitable when classes are unbalanced.
- **Micro**. TP, FN, and FP are calculated globally. When the classes are not mutually exclusive, the micro-PPV or micro-F-score is equal to the accuracy value. Therefore, this metric is useful when there are mutually exclusive classes. 

In this assignment, you are asked to:

1. Develop a function called `confusionMatrix` (same name as in the previous assignment) that returns the values of the metrics adapted to the condition of having more than two classes. To do so, include an additional parameter that allows to calculate them in the *macro* and *weighted* forms.

    This function should receive two matrices: model outputs (`outputs`) and desired outputs (`targets`), both of Boolean elements and dimension 2, with each pattern in a row and each class in a column. The first thing this function should do is to check that the number of columns of both matrices is equal and is different from 2. In case they have only one column, these columns are taken as vectors and the confusionMatrix function developed in the previous assignment is called.
    
    ### Question 4.12
    
    > ❓ Why are two-column matrices invalid?
    
    Because with exactly two classes the problem is binary, so the targets/outputs should be a single Boolean vector (one column) indicating the positive class. A two-column Boolean matrix is redundant—under mutual exclusivity one column is simply the negation of the other—and it can even be internally inconsistent (e.g., both 0 or both 1 in a row). To avoid ambiguity about which column is “positive” and to reuse the binary metrics from the previous assignment, the function rejects the 2-column case and requires either 1 column (binary) or >2 columns (multiclass).
    
    If both matrices have more than 2 columns, the following steps can be followed:
    
    - Reserve memory for the sensitivity, specificity, PPV, NPV and F-score vectors, with one value per class, initially equal to 0. To do this, the `zeros` function can be used.
    
    - Iterate for each class, and, if there are patterns in that class, make a call to the `confusionMatrix` function of the previous assignment, passing as vectors the columns corresponding to the class of that iteration of the outputs and targets matrices. Assign the result to the corresponding element of the sensitivity, specificity, PPV, NPV and $F_1$ vectors.
    - Reserve memory for the confusion matrix.
    - Perform a nested loop where both loops iterate over the class indices to fill in each cell of the confusion matrix.
  ### Question 4.13 
  > 💡 In reality, it is not necessary to reserve memory and use nested loops.  
  > This can be implemented in a more elegant and efficient way using a **comprehension**, in a single line of code.  
  > As a hint, it may help to first write the nested loop with a one-line body, and then try to convert it into a comprehension.

In [2]:
function buildConfusionMatrix(outputs::AbstractArray{Bool,2}, targets::AbstractArray{Bool,2})
    nClasses = size(targets, 2)
    confMatrix = zeros(Int, nClasses, nClasses)

    for i in 1:nClasses
        for j in 1:nClasses
            confMatrix[i, j] = sum(targets[:, i] .& outputs[:, j])
        end
    end

    return confMatrix
end

buildConfusionMatrix (generic function with 1 method)

In [3]:
function buildConfusionMatrix(outputs::AbstractArray{Bool,2}, targets::AbstractArray{Bool,2})
    nClasses = size(targets, 2)
    confMatrix = [sum(targets[:, i] .& outputs[:, j]) for i in 1:nClasses, j in 1:nClasses]
    return confMatrix
end

buildConfusionMatrix (generic function with 1 method)

 
* Aggregate the values of sensitivity, specificity, PPV, NPV, and $F_1$-score for each class into a single value according to the *macro* or *weighed* strategy, as specified in the input argument.
> ⚠️ When using the **weighted** strategy, it is necessary to compute how many instances belong to each class in order to calculate the weighted average.  
> This can be done with `sum(targets, dims=1)`.  
> However, this call returns a matrix with a single row (and one column per class).  
> If you try to compute the element-wise product of this result with one of the metric vectors (e.g., the sensitivity vector), the result will be a matrix, because the metric vector is interpreted as a column — and multiplying a row by a column gives a full matrix.
> ✅ This can be resolved in several ways, the simplest being to **flatten** the matrix of instance counts into a vector using: `vec(sum(targets, dims=1))`
    
* Finally, calculate the accuracy value with the `accuracy` function developed in a previous assignment, and calculate the error rate from this value.

In [4]:
# Safe division
safe_div(a, b) = b == 0 ? NaN : a / b

function confusionMatrix(outputs::AbstractArray{Bool,2}, targets::AbstractArray{Bool,2}; weighted::Bool=true)
    # Basic checks
    size(outputs) == size(targets) || error("outputs and targets must have the same size")
    n, L = size(targets)
    L == size(outputs, 2) || error("outputs and targets must have the same number of columns")

    # If binary (1 column), defer to the previous assignment's function (vector version)
    if L == 1
        return confusionMatrix(vec(outputs), vec(targets))  # assumes the 1D method exists
    elseif L == 2
        error("Two-column matrices are invalid for this function (use the binary version).")
    end

    # Per-class metrics (initialize as NaN so classes with 0 positives don’t skew macro)
    sens = fill(NaN, L)  # sensitivity (recall, TPR)
    spec = fill(NaN, L)  # specificity (TNR)
    ppv  = fill(NaN, L)  # precision (PPV)
    npv  = fill(NaN, L)  # NPV
    f1   = fill(NaN, L)  # F1-score

    # Precompute totals
    N = n

    # Compute per-class metrics
    for i in 1:L
        t = targets[:, i]
        o = outputs[:, i]

        pos = sum(t)
        neg = N - pos

        if pos > 0 || sum(o) > 0
            TP = sum(t .& o)
            TN = sum((.!t) .& (.!o))
            FP = sum((.!t) .& o)
            FN = sum(t .& (.!o))

            sens[i] = safe_div(TP, TP + FN)
            spec[i] = safe_div(TN, TN + FP)
            ppv[i]  = safe_div(TP, TP + FP)
            npv[i]  = safe_div(TN, TN + FN)
            f1[i]   = (isnan(ppv[i]) || isnan(sens[i]) || (ppv[i] + sens[i] == 0)) ? NaN :
                      2 * ppv[i] * sens[i] / (ppv[i] + sens[i])
        end
    end

    # Confusion matrix (rows = true class, cols = predicted class)
    # counts patterns that are true in class i and predicted in class j
    confMatrix = [sum(targets[:, i] .& outputs[:, j]) for i in 1:L, j in 1:L]

    # Accuracy and error rate from confusion matrix
    total = sum(confMatrix)
    correct = sum(diag(confMatrix))
    accuracy = total == 0 ? NaN : correct / total
    error_rate = isnan(accuracy) ? NaN : (1 - accuracy)

    # Aggregation: macro vs weighted
    # Use only classes with at least one positive target to avoid NaNs dominating macro
    counts = vec(sum(targets, dims=1))  # per-class counts (as a vector)
    has_pos = counts .> 0

    agg = function(metric::Vector{Float64})
        if weighted
            w = counts .* .!isnan.(metric)  # zero weight for NaN metrics
            W = sum(w)
            return W == 0 ? NaN : sum(skipmissing(metric .* (w ./ W)))
        else
            vals = metric[has_pos .& .!isnan.(metric)]
            return length(vals) == 0 ? NaN : mean(vals)
        end
    end

    sens_agg = agg(sens)
    spec_agg = agg(spec)
    ppv_agg  = agg(ppv)
    npv_agg  = agg(npv)
    f1_agg   = agg(f1)

    return (accuracy, error_rate, sens_agg, spec_agg, ppv_agg, npv_agg, f1_agg, confMatrix)
end

confusionMatrix (generic function with 1 method)

This function should return a tuple with the same values specified in the previous exercise, in the same order: **accuracy, error rate, sensitivity, specificity, PPV, NPV, F1-score, and the confusion matrix.** As mentioned before, the use of loops is allowed for implementing this function.


3. Develop another function called `confusionMatrix` in which the first parameter `outputs` is of type `AbstractArray{<:Real,2}`, and `targets` is of type `AbstractArray{Bool,2}` (the same as before). What this function should do is to convert the first parameter to an array of boolean values (using the function `classifyOutputs`) and call the previous function. Additionally, this function may receive two optional arguments: `threshold` and `weighted`.  
These values will be used when calling the respective underlying functions that this one builds upon.

  ### Question 4.14
> ❓ Within this function, in which calls will each of these two optional parameters be used?

- The **`threshold`** parameter is used **when calling** the `classifyOutputs` function, since this function converts the real-valued outputs of the model into Boolean values depending on whether they exceed the given threshold.

- The **`weighted`** parameter is used **when calling** the previously defined `confusionMatrix` function (the one that takes Boolean matrices), to indicate whether the performance metrics should be computed using the *macro* or *weighted* strategy.


> ❓ In which cases is the `threshold` parameter required?

The **`threshold`** parameter is required when the model outputs are **continuous real values** (for example, when using linear or sigmoidal activation functions in the output layer).  
In these cases, it is necessary to convert those values into Boolean form to determine whether each output neuron is considered *active* (positive) or *inactive* (negative).  
If the outputs are already Boolean, the `threshold` parameter is **not needed**.


Since this function builds upon the previous one, it **should not contain any loops**.

In [5]:
function confusionMatrix(outputs::AbstractArray{<:Real,2},
                         targets::AbstractArray{Bool,2};
                         threshold::Real=0.5, weighted::Bool=true)

    size(outputs) == size(targets) || error("outputs and targets must have the same size")

    # 1) Convert real-valued outputs to Boolean using the given threshold
    o_bool = classifyOutputs(outputs; threshold=threshold)

    # 2) Delegate to the Boolean-matrix version (no loops here)
    return confusionMatrix(o_bool, targets; weighted=weighted)
end

confusionMatrix (generic function with 2 methods)

4. Override this function once again by developing another function of the same name that performs the same task, but this time taking as inputs two vectors (`targets` and `outputs`) of the same length, whose elements are of any type (i.e., they are of type `AbstractArray{<:Any}`), plus the additional parameter that allows to aggregate the metrics through the *macro* and *weighted* strategies. The vectors `outputs` and `targets` contain the predicted and true labels for each instance, respectively. Therefore, both vectors must have the same length. The vector `classes` contains all possible class labels, and will usually have a different length than `outputs` and `targets`. The elements of these vectors represent labels and may take various forms — such as real numbers, integers, strings, or symbols. For this reason, the type `Any` is used. For example, a valid `classes` vector could be:
   ```julia
       ["dog", "cat", 3, :green]
    ```
    Obviously, both the predicted labels (`outputs` vector) and the true labels (`targets` vector) must be included in the `classes` vector.

  ### Question 4.15
 > ❓ Write this check without using any loops. You may find the functions `all`, `in`, and `unique` useful. (A solution is provided at the end of the notebook.)

The defensive programming check ensures that **all predicted classes** in `outputs` are included among the **possible true classes** in `targets`.  
Without using loops, this can be written as:

```Julia
@assert all(in.(unique(outputs), Ref(unique(targets))))
```

This function can be implemented by calling a previous version of `confusionMatrix`, specifically the one that receives **boolean matrices (2D)** along with an optional argument.

To prepare for this, both `outputs` and `targets` should be **one-hot encoded** by calling the `oneHotEncoding` function, passing the `classes` vector as input.

Once both matrices are encoded, you can call the corresponding `confusionMatrix` function.
⚠️ It is essential to build the `classes` vector **before** the calls to `oneHotEncoding`, and to use the **same vector** in both calls.  

   ### Question 4.16
  > ❓ What could happen if this is not done?

If the `classes` vector is not built **before** calling `oneHotEncoding`, each call could generate a **different encoding scheme** depending on the unique values found in `outputs` or `targets`.  
This would cause the two resulting matrices to have **inconsistent column order or dimensions**, meaning that the same class could correspond to different columns in each matrix.  
As a result, the subsequent comparison in `confusionMatrix` would be incorrect, leading to **misaligned metrics and an invalid confusion matrix**.


In [6]:
function confusionMatrix(outputs::AbstractArray{<:Any,1},
                         targets::AbstractArray{<:Any,1},
                         classes::AbstractArray{<:Any,1};
                         weighted::Bool=true)

    length(outputs) == length(targets) || error("outputs and targets must have the same length")

    # Ensure all labels appear in `classes` (no loops)
    @assert all(in.(unique(outputs), Ref(classes))) "All predicted labels must be in `classes`"
    @assert all(in.(unique(targets), Ref(classes))) "All true labels must be in `classes`"

    # One-hot encode both vectors using the SAME `classes` (no loops)
    o_bool = oneHotEncoding(outputs, classes)
    t_bool = oneHotEncoding(targets, classes)

    # Delegate to the Boolean 2D version (no loops here)
    return confusionMatrix(o_bool, t_bool; weighted=weighted)
end

confusionMatrix (generic function with 3 methods)

Since this function builds on a previous one, it must **return the same outputs** and **must not use loops**.

5. You must now create a final version of `confusionMatrix`, similar to the one described above, but **without receiving the `classes` vector** explicitly. Instead, `classes` should be constructed from `outputs` and `targets` using:
   ```julia
          classes = unique(vcat(targets, outputs))
   ```
  Then, call the previous version of `confusionMatrix`, passing `outputs`, `targets`, `classes`, and the optional argument.

> ⚠️ Use this version with caution.
> If the data has been partitioned (e.g., for cross-validation), some classes might not appear in `outputs` or `targets`, potentially affecting results.

In [7]:
function confusionMatrix(outputs::AbstractArray{<:Any,1},
                         targets::AbstractArray{<:Any,1};
                         weighted::Bool=true)

    length(outputs) == length(targets) || error("outputs and targets must have the same length")

    # Build classes from both vectors (order determined by `unique`)
    classes = unique(vcat(targets, outputs))

    # Delegate to the version that accepts an explicit `classes` vector
    return confusionMatrix(outputs, targets, classes; weighted=weighted)
end


confusionMatrix (generic function with 4 methods)

As before, this function must return the same outputs and should not use loops.

## printConfusionMatrix (multiclass versions)
As in the previous notebook, implement four versions of the `printConfusionMatrix` function — now for the multiclass case, where both `outputs` and `targets` may be matrices or vectors, depending on the context.

In [8]:
using Printf


fmt(x) = isnan(x) ? "NaN" : @sprintf("%.4f", x)

function _print_metrics(acc, err, sens, spec, ppv, npv, f1, cm; weighted::Bool)
    println("=== Multiclass confusion matrix ($(weighted ? "weighted" : "macro")) ===")
    println("Accuracy:     ", fmt(acc))
    println("Error rate:   ", fmt(err))
    println("Sensitivity:  ", fmt(sens))
    println("Specificity:  ", fmt(spec))
    println("PPV:          ", fmt(ppv))
    println("NPV:          ", fmt(npv))
    println("F1-score:     ", fmt(f1))
    println("Confusion matrix (rows = true class, cols = predicted class):")
    show(stdout, "text/plain", cm)
    println()  # newline after matrix
end

_print_metrics (generic function with 1 method)

In [9]:
function printConfusionMatrix(outputs::AbstractArray{Bool,2},
                              targets::AbstractArray{Bool,2};
                              weighted::Bool=true)
    acc, err, sens, spec, ppv, npv, f1, cm = confusionMatrix(outputs, targets; weighted=weighted)
    _print_metrics(acc, err, sens, spec, ppv, npv, f1, cm; weighted=weighted)
    return nothing
end

printConfusionMatrix (generic function with 1 method)

In [10]:
# Uses your Real+Bool confusionMatrix (which internally thresholds & delegates)
function printConfusionMatrix(outputs::AbstractArray{<:Real,2},
                              targets::AbstractArray{Bool,2};
                              weighted::Bool=true)
    acc, err, sens, spec, ppv, npv, f1, cm = confusionMatrix(outputs, targets; weighted=weighted)
    _print_metrics(acc, err, sens, spec, ppv, npv, f1, cm; weighted=weighted)
    return nothing
end

printConfusionMatrix (generic function with 2 methods)

In [11]:
function printConfusionMatrix(outputs::AbstractArray{<:Any,1},
                              targets::AbstractArray{<:Any,1},
                              classes::AbstractArray{<:Any,1};
                              weighted::Bool=true)
    acc, err, sens, spec, ppv, npv, f1, cm = confusionMatrix(outputs, targets, classes; weighted=weighted)
    _print_metrics(acc, err, sens, spec, ppv, npv, f1, cm; weighted=weighted)
    return nothing
end

printConfusionMatrix (generic function with 3 methods)

In [12]:
function printConfusionMatrix(outputs::AbstractArray{<:Any,1},
                              targets::AbstractArray{<:Any,1};
                              weighted::Bool=true)
    acc, err, sens, spec, ppv, npv, f1, cm = confusionMatrix(outputs, targets; weighted=weighted)
    _print_metrics(acc, err, sens, spec, ppv, npv, f1, cm; weighted=weighted)
    return nothing
end

printConfusionMatrix (generic function with 4 methods)

All four versions must also accept an optional argument to choose whether metrics are computed using the macro or weighted strategy.

> 🛠 These four functions will not be graded, but they will be useful for Exercise 2.

### Learn Julia

The defensive programming line to ensure that all classes of the `output` vector are included in the desired output vector is as follows:

```Julia
@assert(all([in(output, unique(targets)) for output in outputs]))
```